### Word Group Videos Select

In [15]:
import os
import re
import sys
import numpy as np
import pandas as pd
import glob
from gtts import gTTS
from PIL import Image, ImageFilter, ImageOps
#from playsound import playsound
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip, concatenate_audioclips,CompositeVideoClip, CompositeAudioClip
import pandas as pd
import nltk
from nltk import word_tokenize
from pathlib import Path
import shutil
from os.path import isfile, join
import time
import cv2

In [16]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

filename_length_num = 5

# lang
if lang_folder == "English":
    lang = 'en'
elif lang_folder == "Turkish":
    lang = 'tr'
elif lang_folder == "German":
    lang = 'de'
elif lang_folder == "French":
    lang = 'fr'
elif lang_folder == "Spanish":
    lang = 'es'
elif lang_folder == "Arabic":
    lang = 'ar'
elif lang_folder == "Portuguese":
    lang = 'pt'
elif lang_folder == "Dutch":
    lang = 'nl'
elif lang_folder == "Italian":
    lang = 'it'
else:
    pass

print(f"{lang_folder}\n{lang}") 

Turkish
tr


In [17]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [18]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Output Path Folder File

In [ ]:
# Output path and folder
result_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words"
main_result_folder = "Deployment3 Select"  # Fourgram_Selected, Test2
result_folder = f"Ngram Video Select"  # Twogram, Threegram, Fourgram, Sample


In [ ]:
# ///***
output_path = f"{result_path}/{main_result_folder}/{result_folder}"  # output path

Path(output_path).mkdir(parents=True, exist_ok=True)
# ****\\\

#### Read Data

In [74]:
group_num = 1 # 1,2,3
# twogram
twogram_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Twogram{group_num}/Result/Concat_Video"
# threegram
threegram_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Threegram{group_num}/Result/Concat_Video"
# fourgram
fourgram_path_folder = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Fourgram{group_num}/Result/Concat_Video"

In [75]:
df_deployment_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/Manuel Select/\
Revised Ngrams All in One.xlsx", sheet_name="Sheet2")
df_deployment_file.drop_duplicates(inplace=True)
df_deployment_file.reset_index(drop=True, inplace=True)
df_deployment_file

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,yardım eder misin,eder misin
2,bana yardım eder misin,yardım eder misin,yardım eder
3,başka bir şey daha,başka bir şey,bir şey
4,başka bir şey daha,başka bir şey,başka bir
...,...,...,...
475,NaN,çok kötü bir,çok kötü
476,NaN,bir şey olmaz,NaN
477,NaN,şuna bir bak,bir bak
478,NaN,şuna bir bak,NaN


In [76]:
df_fourgram_selected_for_course = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
95,her şey iyi olacak,2479
96,öyle bir şey değil,2478
97,bunu sen mi yaptın,2453
98,ne demek oluyor bu,2431


In [77]:
df_deployment_file_freq = pd.merge(df_deployment_file, df_fourgram_selected_for_course, how="left", on="fourgram")
df_deployment_file_freq = df_deployment_file_freq.sort_values(by="frequency",ascending=False)
df_deployment_file_freq.reset_index(drop=True, inplace=True)
df_deployment_file_freq

,fourgram,threegram,twogram,frequency
0,bir şey var mı,bir şey var,NaN,41773.0
1,bir şey var mı,bir şey var,bir şey,41773.0
2,bir şey var mı,NaN,var mı,41773.0
3,bir şey var mı,NaN,şey var,41773.0
4,her şey yolunda mı,NaN,yolunda mı,31126.0
...,...,...,...,...
475,NaN,çok kötü bir,çok kötü,NaN
476,NaN,bir şey olmaz,NaN,NaN
477,NaN,şuna bir bak,bir bak,NaN
478,NaN,şuna bir bak,NaN,NaN


In [78]:
word_count_result(df_deployment_file_freq, ["fourgram","threegram","twogram"], set_condition=True)

,word,word_count
0,bir,124
1,şey,90
2,ne,72
3,var,46
4,bu,36
...,...,...
155,peki,1
156,dur,1
157,pek,1
158,kimsin,1


In [79]:
#df_deployment_file2 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Result/\
#2-1-Word Prefix Suffix Analysis/{lang_folder.capitalize()}_1000_Word_Person_Case_Tense_Plural_Mean_Manuel.xlsx", sheet_name="Sheet1")
#df_deployment_file2.drop_duplicates(inplace=True)
#df_deployment_file2.reset_index(drop=True, inplace=True)
#df_deployment_file2

#### Select Ngram Videos

In [83]:
for i in range(len(df_deployment_file_freq)):
    # read ngram
    fourgram = df_deployment_file_freq.loc[i,"fourgram"]
    threegram = df_deployment_file_freq.loc[i,"threegram"]
    twogram = df_deployment_file_freq.loc[i,"twogram"]
    # make path
    if not pd.isna(fourgram):
        output_path_new = f"{output_path}/Fourgram/{fourgram}"
        Path(output_path_new).mkdir(parents=True, exist_ok=True)
    elif not pd.isna(threegram):
        output_path_new = f"{output_path}/Threegram/{threegram}"
        Path(output_path_new).mkdir(parents=True, exist_ok=True)
    else:
        output_path_new = f"{output_path}/Twogram/{twogram}"
        Path(output_path_new).mkdir(parents=True, exist_ok=True)
    # copy move ngram
    try:
        try:
            source = f"{fourgram_path_folder}/{fourgram}.mp4"
            destination = output_path_new
            shutil.copy2(source, destination)
        except:
            pass
        try:
            source2 = f"{threegram_path_folder}/{threegram}.mp4"
            destination = output_path_new
            shutil.copy2(source2, destination)
        except:
            pass
        try:
            source3 = f"{twogram_path_folder}/{twogram}.mp4"
            destination = output_path_new
            shutil.copy2(source3, destination)
        except:
            pass
    except:
        pass   

In [34]:
type(df_deployment_file_freq.loc[470,"fourgram"])

float

In [40]:
pd.isna(df_deployment_file_freq.loc[1,"fourgram"])

False

In [14]:
range(len(df_deployment_file_freq))

range(0, 480)

In [15]:
df_deployment_file_freq.loc[5,"fourgram"]

'her şey yolunda mı'

### Word Group Videos Select Without Repeat

#### Output Path Folder File2

In [19]:
# Output path and folder
result_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Image Audio Video/Result/1000 Words"
main_result_folder = "Deployment3 Select"  # Fourgram_Selected, Test2
result_folder2 = f"Ngram Video Select2"  # Twogram, Threegram, Fourgram, Sample

In [20]:
# ///***
output_path2 = f"{result_path}/{main_result_folder}/{result_folder2}"  # output path

Path(output_path2).mkdir(parents=True, exist_ok=True)
# ****\\\

#### Read Data2

In [21]:
group_num = 1 # 1,2,3
# twogram
twogram_path_folder2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Twogram{group_num}/Result2/Concat_Video"
# threegram
threegram_path_folder2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Threegram{group_num}/Result2/Concat_Video"
# fourgram
fourgram_path_folder2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Image Audio Video/Result/1000 Words/Deployment3/Fourgram{group_num}/Result2/Concat_Video"

In [22]:
df_deployment_file2 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/Data/Manuel Select/\
Revised Ngrams All in One.xlsx", sheet_name="Sheet2")
df_deployment_file2.drop_duplicates(inplace=True)
df_deployment_file2.reset_index(drop=True, inplace=True)
df_deployment_file2

,fourgram,threegram,twogram
0,bana yardım eder misin,NaN,NaN
1,bana yardım eder misin,yardım eder misin,eder misin
2,bana yardım eder misin,yardım eder misin,yardım eder
3,başka bir şey daha,başka bir şey,bir şey
4,başka bir şey daha,başka bir şey,başka bir
...,...,...,...
475,NaN,çok kötü bir,çok kötü
476,NaN,bir şey olmaz,NaN
477,NaN,şuna bir bak,bir bak
478,NaN,şuna bir bak,NaN


In [23]:
df_fourgram_selected_for_course2 = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment3/\
Data/Manuel Select/Selected 4grams.xlsx")
df_fourgram_selected_for_course2

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
95,her şey iyi olacak,2479
96,öyle bir şey değil,2478
97,bunu sen mi yaptın,2453
98,ne demek oluyor bu,2431


In [24]:
df_deployment_file_freq2 = pd.merge(df_deployment_file2, df_fourgram_selected_for_course2, how="left", on="fourgram")
df_deployment_file_freq2 = df_deployment_file_freq2.sort_values(by="frequency",ascending=False)
df_deployment_file_freq2.reset_index(drop=True, inplace=True)
df_deployment_file_freq2

,fourgram,threegram,twogram,frequency
0,bir şey var mı,bir şey var,NaN,41773.0
1,bir şey var mı,bir şey var,bir şey,41773.0
2,bir şey var mı,NaN,var mı,41773.0
3,bir şey var mı,NaN,şey var,41773.0
4,her şey yolunda mı,NaN,yolunda mı,31126.0
...,...,...,...,...
475,NaN,çok kötü bir,çok kötü,NaN
476,NaN,bir şey olmaz,NaN,NaN
477,NaN,şuna bir bak,bir bak,NaN
478,NaN,şuna bir bak,NaN,NaN


In [25]:
word_count_result(df_deployment_file_freq2, ["fourgram","threegram","twogram"], set_condition=True)

,word,word_count
0,bir,124
1,şey,90
2,ne,72
3,var,46
4,bu,36
...,...,...
155,peki,1
156,dur,1
157,pek,1
158,kimsin,1


#### Select Ngram Videos2

In [26]:
# copy and remove original file
# attention take copy original data folder of files 
for i in range(len(df_deployment_file_freq2)):
    # read ngram
    fourgram = df_deployment_file_freq2.loc[i,"fourgram"]
    threegram = df_deployment_file_freq2.loc[i,"threegram"]
    twogram = df_deployment_file_freq2.loc[i,"twogram"]
    # make path
    if not pd.isna(fourgram):
        output_path_new2 = f"{output_path2}/Fourgram/{fourgram}"
        Path(output_path_new2).mkdir(parents=True, exist_ok=True)
    elif not pd.isna(threegram):
        output_path_new2 = f"{output_path2}/Threegram/{threegram}"
        Path(output_path_new2).mkdir(parents=True, exist_ok=True)
    else:
        output_path_new2 = f"{output_path2}/Twogram/{twogram}"
        Path(output_path_new2).mkdir(parents=True, exist_ok=True)
    # copy move ngram
    try:
        try:
            source = f"{fourgram_path_folder2}/{fourgram}.mp4"
            destination = output_path_new2
            shutil.copy2(source, destination)
            os.remove(source)
        except:
            pass
        try:
            source2 = f"{threegram_path_folder2}/{threegram}.mp4"
            destination = output_path_new2
            shutil.copy2(source2, destination)
            os.remove(source2)
        except:
            pass
        try:
            source3 = f"{twogram_path_folder2}/{twogram}.mp4"
            destination = output_path_new2
            shutil.copy2(source3, destination)
            os.remove(source3)
        except:
            pass
    except:
        pass   

In [ ]:
# copy and remove original file (will test)
# attention take copy original data folder of files 
for i in range(len(df_deployment_file_freq2)):
    # read ngram
    fourgram = df_deployment_file_freq2.loc[i,"fourgram"]
    threegram = df_deployment_file_freq2.loc[i,"threegram"]
    twogram = df_deployment_file_freq2.loc[i,"twogram"]
    # make path
    if not pd.isna(fourgram):
        output_path_new2 = f"{output_path2}/Fourgram/{fourgram}"
        Path(output_path_new2).mkdir(parents=True, exist_ok=True)
    elif not pd.isna(threegram):
        output_path_new2 = f"{output_path2}/Threegram/{threegram}"
        Path(output_path_new2).mkdir(parents=True, exist_ok=True)
    else:
        output_path_new2 = f"{output_path2}/Twogram/{twogram}"
        Path(output_path_new2).mkdir(parents=True, exist_ok=True)
    # copy move ngram
    try:
        try:
            source = f"{twogram_path_folder2}/{twogram}.mp4"
            destination = output_path_new2
            shutil.copy2(source, destination)
            os.remove(source)
        except:
            pass
        try:
            source2 = f"{threegram_path_folder2}/{threegram}.mp4"
            destination = output_path_new2
            shutil.copy2(source2, destination)
            os.remove(source2)
        except:
            pass
        try:
            source3 = f"{fourgram_path_folder2}/{fourgram}.mp4"
            destination = output_path_new2
            shutil.copy2(source3, destination)
            os.remove(source3)
        except:
            pass
    except:
        pass   